# Batch processing of large data sets

For the analysis of large data sets, we recommend parallel batch processing for faster run times. Below are scripts for structural and functional analysis using of multiple tif-files using `multiprocessing`. 

## Structural analysis

Parameters for batch analysis should be optimized on one or few representative tif-files. Details on the analysis of sarcomere structure for a single tif-file can be found [here](tutorial_structure_analysis.ipynb).

In [ ]:
# copy this code in .py file and execute. 

import glob
from multiprocessing import Pool
from sarcasm import *

# select folder with tif files
folder = 'D:/2023_SarcAsM_drugs_chronic/'

# find all tif files in folder
tif_files = glob.glob(folder + '*/*.tif')
print(f'{len(tif_files)} tif-files found')

# function for analysis of single tif-file
def analyze_tif(file):
    print(file)
    # initialize SarcAsM object
    sarc_obj = SarcAsM(file)

    # detect sarcomere z-bands, m-bands, sarcomere orientation and cell masks
    sarc_obj.structure.detect_sarcomeres(max_patch_size=(2048, 2048))

    # analyze sarcomere structures (or use step-by-step analysis, see tutorial structure analysis)
    sarc_obj.structure.full_analysis_structure(frames='all')

    print(f'{file} successfully analyzed!')


# set number of pools
n_pools = 3

if __name__ == '__main__':
    with Pool(n_pools) as p:
        p.map(analyze_tif, tif_files)

## Motion analysis

Parameters for batch analysis should be optimized on one or few representative tif-files. Details on the functional analysis of a single movie of a beating cardiomyocyte can be found [here](tutorial_motion_analysis.ipynb).

In [ ]:
# copy this code in .py file and execute. 

from multiprocessing import Pool
from sarcasm import *

folder = 'D:/SarcAsM_drugs/'

# find files
files = glob.glob(folder + '/*/*.tif')
print(f'{len(files)} tif-files found')

# detect LOIs
def detect_lois(file):
    # initialize file
    sarc_obj = SarcAsM(file)

    # detect all sarcomere features for first frame only
    sarc_obj.structure.detect_sarcomeres(frames=0)

    # detect Z-bands for all frames with time-consistent 3D-U-Net
    sarc_obj.structure.detect_z_bands_fast_movie()

    # analyze sarcomere vectors in first frame
    sarc_obj.structure.analyze_sarcomere_vectors(frames=0)

    # detect lines of interest (LOIs)
    sarc_obj.structure.detect_lois(n_lois=4)

# analyze all LOIs of one tif-file
def analyze_lois(file):
    lois = Utils.get_lois_of_file(file)
    for file, loi in lois:
        try:
            # initialize LOI
            mot_obj = Motion(file, loi)

            # analysis of LOI with default parameters
            mot_obj.full_analysis_loi()

        except Exception as e:
            print(file, loi)
            print(repr(e))


if __name__ == '__main__':
    # find LOIs
    with Pool(4) as p:
        p.map(detect_lois, files)
    
    # analyze LOIs
    with Pool(12) as p:
        p.map(analyze_lois, files)